In [1]:
import niimpy
import niimpy.preprocessing.location
import niimpy.analysis.social_signature
import pandas
import folium

In [2]:
data = niimpy.reading.google_takeout.location_history(
    "takeout-20241111T082655Z-001.zip",
    start_date = pandas.to_datetime("2022-01-01", utc=True),
    end_date = pandas.to_datetime("2022-06-30", utc=True),
)
data = niimpy.preprocessing.location.filter_location(
    data,
    remove_disabled=False, remove_network=False, remove_zeros=True
)
data.head(5)

Index(['accuracy', 'source', 'device', 'altitude', 'verticalaccuracy',
       'platformtype', 'servertimestamp', 'devicetimestamp', 'batterycharging',
       'formfactor', 'velocity', 'heading', 'placeid', 'latitude', 'longitude',
       'inferred_latitude', 'inferred_longitude', 'activity_type',
       'activity_inference_confidence', 'user'],
      dtype='object')

In [4]:
niimpy.preprocessing.location.location_significant_place_features(data)

,n_rare,n_top5,n_top2,device,n_moving,user,n_transitions,n_static,n_top3,n_top4,normalized_entropy,n_sps,max_dist_home,n_top1,n_home,entropy
timestamp,,,,,,,,,,,,,,,,
2022-04-30 00:00:00+00:00,2.0,0.0,31.0,1832214273,31.0,92356d32-317a-11f0-87b0-b906c1020b08,5.0,136.0,11.0,2.0,4.330821,3.0,5353.427305,92.0,91.0,4.757893
2022-05-31 00:00:00+00:00,19.0,74.0,543.0,1832214273,1477.0,92356d32-317a-11f0-87b0-b906c1020b08,81.0,4343.0,200.0,94.0,2.423967,16.0,40346.976685,3126.0,141.0,6.720663
2022-06-30 00:00:00+00:00,70.0,82.0,302.0,1832214273,1992.0,92356d32-317a-11f0-87b0-b906c1020b08,164.0,3172.0,136.0,106.0,2.155481,37.0,605332.009054,1770.0,170.0,7.783265


In [5]:
niimpy.preprocessing.location.find_home(data["latitude"], data["longitude"], data.index)

(np.float64(60.22989181622235), np.float64(24.77360194254112))

In [ ]:
data.loc["2022-05-15 12:00:00":"2022-05-15 22:45:00"]

,accuracy,source,device,altitude,verticalaccuracy,platformtype,servertimestamp,devicetimestamp,batterycharging,formfactor,velocity,heading,placeid,latitude,longitude,inferred_latitude,inferred_longitude,activity_type,activity_inference_confidence,user
timestamp,,,,,,,,,,,,,,,,,,,,
2022-05-15 19:43:46.907000+00:00,12.0,WIFI,1832214273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.230027,24.773754,NaN,NaN,NaN,NaN,92356d32-317a-11f0-87b0-b906c1020b08
2022-05-15 20:20:40.888000+00:00,12.0,WIFI,1832214273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.230027,24.773754,NaN,NaN,NaN,NaN,92356d32-317a-11f0-87b0-b906c1020b08
2022-05-15 21:00:00.888000+00:00,12.0,WIFI,1832214273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.230027,24.773754,NaN,NaN,NaN,NaN,92356d32-317a-11f0-87b0-b906c1020b08
2022-05-15 21:36:05.844000+00:00,12.0,WIFI,1832214273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.230027,24.773754,NaN,NaN,NaN,NaN,92356d32-317a-11f0-87b0-b906c1020b08
2022-05-15 21:45:50.830000+00:00,15.0,WIFI,1832214273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.230036,24.773764,NaN,NaN,NaN,NaN,92356d32-317a-11f0-87b0-b906c1020b08


In [90]:
day_data = data.loc["2022-05-15 12:10:00":"2022-05-15 15:50:00"]
locations = day_data[["latitude", "longitude"]].drop_duplicates()
map_center = [locations['latitude'].mean(), locations['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

folium.PolyLine(locations[['latitude', 'longitude']].values, color="blue", weight=2.5, opacity=1).add_to(mymap)

for _, row in locations.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(mymap)

mymap

In [ ]:
mail_data = niimpy.reading.google_takeout.email_activity(
    "takeout-20241111T082655Z-001.zip",
    start_date = pandas.to_datetime("2022-10-01", utc=True),
)
mail_data

Failed to decode message: decode() argument 'encoding' must be str, not None


,received,from,to,cc,bcc,message_id,in_reply_to,character_count,word_count,message_type,user
timestamp,,,,,,,,,,,
2024-11-09 09:59:28+00:00,2024-11-09 01:59:29-08:00,829,[0],[],[],2718,3685,607,57,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
2024-11-04 10:29:41+00:00,2024-11-04 02:29:42-08:00,147,[0],[],[],3529,<NA>,2225,262,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
2024-11-01 06:44:12+00:00,2024-10-31 23:44:13-07:00,147,[0],[],[],437,<NA>,1336,146,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
2024-10-31 00:39:39+00:00,2024-10-30 17:39:42-07:00,118,[0],[],[],3367,<NA>,1735,170,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
2024-10-29 15:42:27+00:00,2024-10-29 08:42:29-07:00,428,[0],[],[],904,2301,403,13,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-14 01:40:06+00:00,2023-07-13 18:41:11-07:00,751,[81],[],[],727,4129,461,73,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
2024-08-10 14:20:10+00:00,2024-08-10 07:20:24-07:00,562,"[856, 562]",[],[],1661,2175,6370,891,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18
2024-07-11 18:00:32+00:00,2024-07-11 11:00:53-07:00,404,[404],[],[],565,2708,2269,213,incoming,f4dac0a8-3153-11f0-a49e-2d9cee4c7e18


In [ ]:
social_signature = niimpy.analysis.social_signature.social_signature(mail_data)
social_signature.head(10)

to
21     0.027778
280    0.055556
323    0.083333
324    0.027778
440    0.027778
452    0.027778
528    0.027778
584    0.055556
671    0.166667
734    0.055556
dtype: float64

In [ ]:
social_signature = niimpy.analysis.social_signature.social_signature(mail_data, include_received=False)
social_signature.head(10)

TypeError: social_signature() got an unexpected keyword argument 'include_received'